# Movie Recommendation System - Neural Collaborative Filtering

# Load Required Data

In [ ]:
import numpy as np
import pandas as pd

# Load the movie embeddings and movie metadata
movie_embedding_gmf = np.load('/content/drive/MyDrive/movie_embedding_gmf_backup.npy')
df_movies = pd.read_csv('/content/drive/MyDrive/movies_backup.csv')

# Create a list of movie titles
movie_list = df_movies['title'].tolist()

# Define the Prediction Function

In [ ]:
def predict(movie_ids, ratings, movie_embedding_layer):
    """
    Predict top 10 movie recommendations based on user-rated movies.

    Parameters:
    - movie_ids: List of indices corresponding to the movies rated by the user.
    - ratings: List of ratings provided by the user for the selected movies.
    - movie_embedding_layer: Numpy array containing movie embeddings.

    Returns:
    - List of indices corresponding to the top 10 recommended movies.
    """
    # Retrieve embeddings for the selected movies
    movie_embeddings = movie_embedding_layer[movie_ids]

    # Compute the new user embedding as the weighted average of selected movie embeddings
    new_user_embedding = np.mean(movie_embeddings * np.array(ratings).reshape(-1, 1), axis=0)

    # Compute predicted ratings for all movies
    predicted_ratings = np.dot(movie_embedding_layer, new_user_embedding)

    # Get indices of top 10 movies with highest predicted ratings
    top_movie_ids = np.argsort(-predicted_ratings)[:10]

    return top_movie_ids

# Interactive Movie Rating and Recommendation

In [ ]:
import random
import ipywidgets as widgets
from IPython.display import display, clear_output

# Randomly select 10 movies for the user to rate
selected_movies = random.sample(movie_list, 10)

# Create sliders for user to rate the selected movies
rating_sliders = {}
print("Please rate the following movies (0.0 to 10.0):\n")
for movie in selected_movies:
    slider = widgets.FloatSlider(value=5.0, min=0.0, max=10.0, step=0.5, description=movie, continuous_update=False)
    rating_sliders[movie] = slider
    display(slider)

# Button to get recommendations
recommend_button = widgets.Button(description="Get Recommendations")

# Output area to display recommendations
output = widgets.Output()

def on_recommend_button_clicked(b):
    with output:
        clear_output()
        # Retrieve user ratings
        user_ratings = {movie: slider.value for movie, slider in rating_sliders.items()}

        # Filter out movies with zero rating
        rated_movies = [movie for movie, rating in user_ratings.items() if rating > 0]
        ratings = [user_ratings[movie]/10 for movie in rated_movies]  # Normalize ratings to 0-1

        if not rated_movies:
            print("Please rate at least one movie to get recommendations.")
            return

        # Get indices of rated movies
        rated_movie_indices = [df_movies[df_movies['title'] == movie].index[0] for movie in rated_movies]

        # Predict top 10 movie recommendations
        top_movie_ids = predict(
            movie_ids=rated_movie_indices,
            ratings=ratings,
            movie_embedding_layer=movie_embedding_gmf
        )

        # Display recommended movies
        print("\nRecommended Movies for You:\n")
        for idx in top_movie_ids:
            title = df_movies.iloc[idx]['title']
            print(f"- {title}")

# Attach the click event handler to the button
recommend_button.on_click(on_recommend_button_clicked)

# Display the button and output area
display(recommend_button, output)

Please rate the following movies (0.0 to 10.0):



FloatSlider(value=5.0, continuous_update=False, description='Serving Life (2011)', max=10.0, step=0.5)

FloatSlider(value=5.0, continuous_update=False, description='Virtual Combat (1995)', max=10.0, step=0.5)

FloatSlider(value=5.0, continuous_update=False, description='Zindagi Na Milegi Dobara (2011)', max=10.0, step=…

FloatSlider(value=5.0, continuous_update=False, description='Oliver & Company (1988)', max=10.0, step=0.5)

...

FloatSlider(value=5.0, continuous_update=False, description="Everything's Cool (2007)", max=10.0, step=0.5)

FloatSlider(value=5.0, continuous_update=False, description='The Light of Freedom (2013)', max=10.0, step=0.5)

FloatSlider(value=5.0, continuous_update=False, description='Perry Mason: The Case of the Lethal Lesson (1989)…

Button(description='Get Recommendations', style=ButtonStyle())

Output()